In [1]:
import pydicom
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
import importlib
import sys
import cv2
import SimpleITK as sitk

In [2]:
sys.path.append(r'D:\tensorflow\LiChen\FRAPPE')
import DB
importlib.reload(DB)
from DB import DB
import CASCADE
importlib.reload(CASCADE)
from CASCADE import CASCADE
import glob

TPS = ['0','12','18','24','30','36','48','60','72','84','96']

def getdcmpath(pid,TPid,side='L'):
    #following TPs
    VFVersion = '29'
    paths = []
    if TPid not in [0,1,2,3,4,5,6,8,10]:
        print('not valid TP')
        return 
        
    dbconfig = {}
    dbconfig['dbname'] = 'ahaknee'+TPS[TPid]+'tp'+VFVersion
    dbconfig['host']="128.208.221.46"#Server #4
    dbconfig['user']="root"
    dbconfig['passwd']="123456"
    db = DB(config=dbconfig)

    db.getcursor()
    sql = '''\
    SELECT
    pid,eid,dicompath
    FROM
    stat
    where pid = %s
    '''
    db.mycursor.execute(sql,[pid])
    dbr = db.mycursor.fetchall() 
    for di in dbr:
        if di[1][-1]==side:
            return di[2]
    return

In [2]:
def drawcont(cas, shape, fi):
    vesmask = np.zeros(shape,dtype=np.uint8)
                        
    cont = cas.getcontour('OuterWall',fi+1)
    if cont is None or np.size(cont[0])<2:
        print('No lumen contour for frappe dcm slice',fi)
        return vesmask      
    
    intcont = []
    for conti in range(len(cont)):
        intcont.append([int(round(cont[conti][0])),int(round(cont[conti][1]))])
    intcont = np.array(intcont)	
    cv2.fillPoly(vesmask, pts=[intcont], color=(1,1,1))
    return vesmask   

In [5]:
def flip_rl(img):

    # flip the image
    flip_direction = [True, False, False]
    img = sitk.Flip(img, flip_direction)

    # put direction back to identity (direction(0,0) becomes -1 after flip)
    direction = []
    direction.append(1.0)
    direction.append(0.0)
    direction.append(0.0)
    direction.append(0.0)
    direction.append(1.0)
    direction.append(0.0)
    direction.append(0.0)
    direction.append(0.0)
    direction.append(1.0)
    img.SetDirection(direction)
    
    img.SetOrigin((0,0,0))

    return img

In [4]:
# process list
# first in TP as reference
caselist = [
    {'pid': '9189303','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'}
]
    

split_seq_path = r'../../original'

if not os.path.exists(split_seq_path):
    os.mkdir(split_seq_path)
    
vesmaskpath = r'../../vesmask'

In [24]:
#split seqs for a case
    
for casei in caselist:
    pi = casei['pid']
    side = casei['side']
    
    precasepath = split_seq_path + '/' + pi + side + '/'
    if not os.path.exists(precasepath):
        os.mkdir(precasepath)
        
    regtp = casei['TP']
    regtp2 = casei['TP2']
    
    for tpi in range(len(regtp)):
        casepath = getdcmpath(pi, regtp[tpi], side)
            
        if casepath is None:
            print('cannot find dcm path for TPid',regtp[tpi])
            continue
        else:
            print('dcm path for TPid', regtp[tpi], casepath)
            
            tp_precasepath = precasepath + 'TP' + str(regtp[tpi]) + '/'
            if not os.path.exists(tp_precasepath):
                os.mkdir(tp_precasepath)
                
            filelist = os.listdir(casepath)
            for f in filelist:
                if ('.dcm') not in f:
                    del filelist[filelist.index(f)]
                    
            eidir = r'X:/'+str(regtp2[tpi])+'tp29\cascade/'+str(regtp2[tpi])+'/'+pi+'/*'+side # In \\128.208.221.24\OAI-Derived-Data-and-Results
            glb = glob.glob(eidir)
            if len(glb)==0:
                print('No ei')
                continue
                
            ei = os.path.basename(glb[0])
            casname = 'E'+ei+'_'+side
            cascadepath = r'X:/'+str(regtp2[tpi])+'tp29\cascade/'+str(regtp2[tpi])+'/'+pi+'/'+ei
            cas = CASCADE(casname,cascadepath,cascadepath,0)
    
            #very ugly here
            vesmaskstack = []
            for fi in range(len(filelist)):
                src = casepath + '/' + filelist[fi]
                
                srcdcm = pydicom.dcmread(src)
                if abs(float(srcdcm.ImageOrientationPatient[0]))<abs(float(srcdcm.ImageOrientationPatient[1])):
                    #coronal slice showing scan areas
                    print('coronal',src)
                    continue
                
                for attempt in range(20):
                    try:
                        dst = tp_precasepath + filelist[fi]
                        if not os.path.exists(dst):
                            shutil.copyfile(src, dst)
                            print('\rcopying', src, end='')
                    except:
                        print('@@@@Connection loss. Retry!@@@@')
                        continue
                    break
                else:
                    print('###Tried 20 times, still failed')
                    
                vesmask = drawcont(cas, srcdcm.pixel_array.shape, fi)
                vesmaskstack.append(vesmask)
            
            vesmaskstack = np.array(vesmaskstack,dtype=np.uint8)
                
            vesmasksitk = sitk.GetImageFromArray(vesmaskstack)
            tmpsrcdcm = pydicom.dcmread(casepath + '/' + filelist[len(filelist)//2])
            vesmasksitk.SetSpacing((tmpsrcdcm[0x0028,0x0030].value[0],tmpsrcdcm[0x0028,0x0030].value[1],tmpsrcdcm[0x0018,0x0050].value))
            
            if side == 'R':
                vesmasksitk = flip_rl(vesmasksitk)
            
            sitk.WriteImage(vesmasksitk, vesmaskpath + '/'+pi+side+'_TP'+str(regtp[tpi])+'_prep_fv.mha')

ahaknee0tp29  exist
dcm path for TPid 0 W:/OAI Baseline/ge0/9189303/10397606
Loading QVJ X:/0tp29\cascade/0/9189303/10397606L\E10397606L_L.QVJ
Adding QVS E10397606LS101_L.QVS
coronal W:/OAI Baseline/ge0/9189303/10397606/001.dcm
copying W:/OAI Baseline/ge0/9189303/10397606/076.dcmno such contour OuterWall03/10397606/007.dcm W:/OAI Baseline/ge0/9189303/10397606/051.dcm
No lumen contour for frappe dcm slice 75
ahaknee12tp29  exist
dcm path for TPid 1 W:/OAI 12 Months/2.C.2/9189303/20060522/11151205
Loading QVJ X:/12tp29\cascade/12/9189303/20060522L\E20060522L_L.QVJ
Adding QVS E20060522LS101_L.QVS
coronal W:/OAI 12 Months/2.C.2/9189303/20060522/11151205/001.dcm
copying W:/OAI 12 Months/2.C.2/9189303/20060522/11151205/076.dcmno such contour OuterWall189303/20060522/11151205/007.dcm
No lumen contour for frappe dcm slice 75
ahaknee18tp29  exist
cannot find dcm path for TPid 2
ahaknee24tp29  exist
cannot find dcm path for TPid 3
ahaknee30tp29  exist
cannot find dcm path for TPid 4
ahaknee36tp2

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, fixed

# Callback invoked by the IPython interact method for scrolling and modifying the alpha blending
# of an image stack of two images that occupy the same physical space. 
def display_images0(image_z, image):
    img = image[image_z,:,:]
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()
    
def display_images2(image_z, image):
    img = image[:,:,image_z]
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

In [ ]:
interact(display_images0, 
         image_z=(0,vesmasksitk.GetSize()[0]-1), 
         image=fixed(vesmasksitk));